In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime
import dask.dataframe as dd
import dask.array as da
import time
import statsmodels.api as sm
import statistics

In [2]:
new_afterlink = pd.read_csv('link_list1.csv')
new_afterlink

,date,COMNAM,month,year,Mkt-RF,SMB,HML,excess_return
0,1996-01-02,ENERGY WEST INC,1,1996,0.59,-0.31,-0.21,-0.045667
1,1996-01-03,ENERGY WEST INC,1,1996,-0.08,-0.26,0.50,0.022096
2,1996-01-04,ENERGY WEST INC,1,1996,-0.82,-0.38,0.22,-0.058474
3,1996-01-05,ENERGY WEST INC,1,1996,-0.15,0.69,-0.12,-0.060096
4,1996-01-08,ENERGY WEST INC,1,1996,0.24,-0.20,-0.11,-0.019000
...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,6,2016,0.60,0.67,0.31,-0.001972
5721818,2016-06-07,OPOWER INC,6,2016,0.12,0.16,0.10,-0.000027
5721819,2016-06-08,OPOWER INC,6,2016,0.37,0.45,-0.09,-0.001000
5721820,2016-06-09,OPOWER INC,6,2016,-0.27,-0.40,-0.24,-0.001000


In [3]:
new_afterlink['IVOL'] = None
new_afterlink

,date,COMNAM,month,year,Mkt-RF,SMB,HML,excess_return,IVOL
0,1996-01-02,ENERGY WEST INC,1,1996,0.59,-0.31,-0.21,-0.045667,None
1,1996-01-03,ENERGY WEST INC,1,1996,-0.08,-0.26,0.50,0.022096,None
2,1996-01-04,ENERGY WEST INC,1,1996,-0.82,-0.38,0.22,-0.058474,None
3,1996-01-05,ENERGY WEST INC,1,1996,-0.15,0.69,-0.12,-0.060096,None
4,1996-01-08,ENERGY WEST INC,1,1996,0.24,-0.20,-0.11,-0.019000,None
...,...,...,...,...,...,...,...,...,...
5721817,2016-06-06,OPOWER INC,6,2016,0.60,0.67,0.31,-0.001972,None
5721818,2016-06-07,OPOWER INC,6,2016,0.12,0.16,0.10,-0.000027,None
5721819,2016-06-08,OPOWER INC,6,2016,0.37,0.45,-0.09,-0.001000,None
5721820,2016-06-09,OPOWER INC,6,2016,-0.27,-0.40,-0.24,-0.001000,None


In [84]:
#choose just one stock to test.
test_reg = new_afterlink[new_afterlink['COMNAM'] == 'ENERGY WEST INC']
type(test_reg.iloc[0]['COMNAM'])

str

In [86]:
for i in list(test_reg['year'].unique()):
    a = test_reg[test_reg['year'] == i]
    for j in list(a['month'].unique()):
        b = a[a['month'] == j]
        #a.loc[a.month == j, 'IVOL'] = j
        #print(a.loc[a.month == j]['IVOL'])
        x = b[['Mkt-RF', 'SMB', 'HML']]
        y = b['excess_return']
        x = sm.add_constant(x)
        model = sm.OLS(y, x).fit()
        residuals = model.resid
        std = np.std(residuals)
        #test_reg.loc[(test_reg.year == i)&(test_reg.month == j), 'IVOL'] = std
        #test_reg.loc[0:3405].loc[test_reg.month == j, 'IVOL'] = std
        values = test_reg.iloc[0]['COMNAM'] + '-' + str(i) + '-' + str(j) + '-' + str(std)
        #print(values)
        ivol_list.append(values)
#Pass the test, next we go through all the firm stocks in the list_new.

In [88]:
ivol_list = []

In [4]:
return_daily = pd.read_csv("question5.csv")#2.55GB
list_total = list(return_daily['COMNAM'].unique())
list_new = list_total[:2731]

In [90]:
start = time.time()
ini_index = last_index = 0 
for ele in list_new:  
    for in_dex, row in new_afterlink.iloc[ini_index:].iterrows():
        if row['COMNAM'] != ele:
            last_index = in_dex
            break
    if ini_index == last_index:
        last_index = new_afterlink.index[-1]
    test_stock = new_afterlink.iloc[ini_index:last_index+1]
    for j in list(test_stock['year'].unique()):  
            a = test_stock[test_stock['year'] == j]
            for i in list(a['month'].unique()): 
                b = a[a['month'] == i]
                x = b[['Mkt-RF', 'SMB', 'HML']]
                y = b['excess_return']
                x = sm.add_constant(x)
                model = sm.OLS(y, x).fit()
                residuals = model.resid
                std = np.std(residuals)
                #new_afterlink.loc[(new_afterlink.year == j)&(new_afterlink.month == i), 'IVOL'] = std
                values = test_stock.iloc[0]['COMNAM'] + '-' + str(j) + '-' + str(i) + '-' + str(std)
                ivol_list.append(values)
                
    ini_index = last_index

end = time.time()
print(end - start)

1602.9565148353577


In [95]:
ivol_list

['ENERGY WEST INC-1996-1-0.03360984672183263',
 'ENERGY WEST INC-1996-2-0.046149998807133906',
 'ENERGY WEST INC-1996-3-0.026057215818000825',
 'ENERGY WEST INC-1996-4-0.022273497910994523',
 'ENERGY WEST INC-1996-5-0.019078670279231904',
 'ENERGY WEST INC-1996-6-0.025454485074715927',
 'ENERGY WEST INC-1996-7-0.02199972137563161',
 'ENERGY WEST INC-1996-8-0.028546611566044933',
 'ENERGY WEST INC-1996-9-0.026086087256267446',
 'ENERGY WEST INC-1996-10-0.031211661629937135',
 'ENERGY WEST INC-1996-11-0.027833186227572037',
 'ENERGY WEST INC-1996-12-0.038450748651229695',
 'ENERGY WEST INC-1997-1-0.033952503004797144',
 'ENERGY WEST INC-1997-2-0.02941177215148115',
 'ENERGY WEST INC-1997-3-0.03882763360840019',
 'ENERGY WEST INC-1997-4-0.028905392276936313',
 'ENERGY WEST INC-1997-5-0.026690154868133142',
 'ENERGY WEST INC-1997-6-0.02148725136446096',
 'ENERGY WEST INC-1997-7-0.03307958161582588',
 'ENERGY WEST INC-1997-8-0.019935566128633557',
 'ENERGY WEST INC-1997-9-0.0192892651346752

In [108]:
ivol = pd.DataFrame(ivol_list)
ivol.columns = ["total"]
new_ivol = ivol['total'].str.split('-', expand = True)

In [111]:
new_ivol.columns = ["COMNAM", "year", "month", "IVOL", "delete"]
new_ivol = new_ivol.drop(labels = "delete", axis = 1)
new_ivol

,COMNAM,year,month,IVOL
0,ENERGY WEST INC,1996,1,0.03360984672183263
1,ENERGY WEST INC,1996,2,0.046149998807133906
2,ENERGY WEST INC,1996,3,0.026057215818000825
3,ENERGY WEST INC,1996,4,0.022273497910994523
4,ENERGY WEST INC,1996,5,0.019078670279231904
...,...,...,...,...
225983,OPOWER INC,2016,2,0.025192660275210634
225984,OPOWER INC,2016,3,0.040826946537524486
225985,OPOWER INC,2016,4,0.02880164446293606
225986,OPOWER INC,2016,5,0.061190104411635984


In [112]:
new_ivol.to_csv("ivol_list1.csv", index = False)